# dynamische interactie

Na sprint 2 van het ontwikkelen van de TBCI kwam de vraag of ook de code dynamischer gebruikt kan woorden:

In [1]:
from pathlib import Path

from toolbox_continu_inzicht import Config

## 1. Voorbeeld met CSV
Lees de configuratie in:

Dit leest een configuratie bestand `test_config.yaml` in. 
Hier definieren we een aantal data adpaters die lezen/schrijven mogelijk maken:

```yaml
GlobalVariables:
    rootdir: 'tests/src/base/data_sets'

DataAdapter:
    postgresql_database:
        database: 'citoolbox'
        schema: 'citoolbox_schema'
        
    my_csv_in: 
        type: csv
        file: 'test_csv_in.csv'
    my_csv_in_delim: 
        type: csv
        file: 'test_csv_in_delim.csv'
        sep: ';'
    MyCSV_out: 
        type: csv
        file: 'test_csv_out.csv'
    MyPostgresql: 
        type: postgresql_database
        table: data_test
    MyNetCDF_in:
        type: netcdf
        file: 'test_netcdf_in.nc'
    MyNetCDF_out:
        type: netcdf
        file: 'test_netcdf_out.nc'

...

```

In [2]:
test_data_sets_path = Path.cwd() / "data_sets"
config = Config(config_path=test_data_sets_path / "test_config.yaml")
config.lees_config()

Zet de data adapter klaar en geef deze de configuratie mee 

In [3]:
from toolbox_continu_inzicht import DataAdapter

data_adapter = DataAdapter(config=config)

Met de functie `set_global_variable` kan dit nu

In [4]:
data_adapter.config.global_variables["calc_time"]

datetime.datetime(2024, 11, 18, 13, 0, tzinfo=datetime.timezone.utc)

In [5]:
from datetime import datetime, timedelta, timezone

dt_new_time = datetime.now() + timedelta(days=1)
new_time = datetime(
    dt_new_time.year,
    dt_new_time.month,
    dt_new_time.day,
    dt_new_time.hour,
    0,
    0,
).replace(tzinfo=timezone.utc)

data_adapter.set_global_variable("calc_time", new_time)

In [6]:
data_adapter.config.global_variables["calc_time"]

datetime.datetime(2024, 11, 19, 14, 0, tzinfo=datetime.timezone.utc)

In [7]:
from toolbox_continu_inzicht.proof_of_concept import example_module

keer_twee = example_module.ValuesTimesTwo(data_adapter=data_adapter)

In [8]:
keer_twee.run(input="mycsv_in", output="mycsv_out")

In [9]:
input_df = keer_twee.df_in

In [10]:
input_df

,objectid,objecttype,parameterid,date_time,value
0,1,measuringstation,1,1726227377000,4.8
1,2,measuringstation,1,1726227377000,5.0
2,3,measuringstation,1,1726227377000,5.2
3,4,measuringstation,1,1726227377000,5.4
4,5,measuringstation,1,1726227377000,5.6


```yaml
GlobalVariables:
    rootdir: 'data_sets'
    moments: [-24,0,24,48] 

DataAdapter:
    default_options:        
        csv:
            sep: ","

    my_csv: 
        type: csv
        file: "test_csv_in.csv"
        sep: ";"
    my_df_python:
        type: python
    my_csv_out:   
        type: csv
        file: "test_csv_in.csv"

```

In [11]:
config = Config(config_path=test_data_sets_path / "test_config_dynamic.yaml")
config.lees_config()

In [12]:
data_adapter = DataAdapter(config=config)
data_adapter.set_dataframe_adapter("my_df_python", input_df)


In [20]:
data_adapter = DataAdapter(config=config)
data_adapter.set_dataframe_adapter("wrong_input_key", input_df,if_not_exist="wrong_input_str")

UserWarning: Data adapter `key='wrong_input_key'` niet gevonden, en if_not_exist='wrong_input_str' is ongeldig, moet `raise` of `create` zijn

In [13]:
keer_twee = example_module.ValuesTimesTwo(data_adapter=data_adapter)
keer_twee.run(input="my_df_python", output="my_df_python")

In [14]:
df_in_functie2 = keer_twee.df_out
df_in_functie2

,objectid,objecttype,parameterid,date_time,value
0,1,measuringstation,1,1726227377000,9.6
1,2,measuringstation,1,1726227377000,10.0
2,3,measuringstation,1,1726227377000,10.4
3,4,measuringstation,1,1726227377000,10.8
4,5,measuringstation,1,1726227377000,11.2


In [15]:
data_adapter.set_dataframe_adapter(
    "my_df_python_2", df_in_functie2, if_not_exist="create"
)

In [18]:
data_adapter.config.data_adapters.keys()

dict_keys(['default_options', 'my_csv', 'my_df_python', 'my_csv_out', 'my_df_python_2'])